In [ ]:
import pandas as pd
import numpy as np
from numpy import arange, array, ones

import math 

from tqdm import trange, tqdm

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pylab

import scipy
import scipy.cluster.hierarchy as shc
from scipy.optimize import curve_fit

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn import preprocessing

plt.style.use('ggplot')

### Чтение базы данных

In [ ]:
#get names for gauges

my_databse = r'D:\science things\Izhora_Plateu\Data\Hydro\Base.xlsx'

xlsx = pd.ExcelFile(my_databse)

WS_area_LIST = [784, 94, 505, 573, 53, 192, 920, 206, 581, 316, 544, 948]
WS_Names = xlsx.sheet_names 
WS = dict(zip(WS_Names, WS_area_LIST))

### Функции

In [ ]:
## Чтение файла с листа EXCEL

def from_df_to_discharge_chaos(df):
    ex1 = df.to_numpy()

    s = ex1.shape

    for i in range(s[1]):
        ex1[:, i] = pd.to_numeric(ex1[:, i], errors='coerce')

    ex1 = ex1.astype(float)

    a = list(range(1, s[1], 14)) # from left to right
    a = a[0:2]
    b = list(range(0, s[0], 34)) # from top to bottom

    # assign years through list

    test = []
    for j in b:
        for i in a:
            test.append(ex1[j+2:j+33, i:i + 12].T)
    test = test[1:]
    return test

## Определение лишних NaN в данных

def days_in_month(month, year):
    def check_days(year):
        if year % 4 != 0 or (year % 100 == 0 and year % 400 != 0):
            return False
        else:
            return True    
    if month in {1, 3, 5, 7, 8, 10, 12}:
        return 31
    if month == 2:
        if check_days(year):
            return 29
        else:
            return 28
    return 30

## Ровняем все месяцы, с учётом NaN

def make_monthes_cute(test):
    finalized_discharge_data = [[[] for i in range(len(test[j]))] for j in range(len(test))]
    for i, clear_year in enumerate(finalized_discharge_data):
        for j, month in enumerate(test[i]):
            if len(month) == days_in_month(monthes_for_years[i][j].month, monthes_for_years[i][j].year):
                clear_year[j] = test[i][j]
            else:
                clear_year[j] = test[i][j][:days_in_month(monthes_for_years[i][j].month, monthes_for_years[i][j].year)]
    return finalized_discharge_data

## Приводим к длинным рядам

def to_long_series(finalized_discharge_data):
    daily_discharges = [item for sublist in [item for sublist in finalized_discharge_data for item in sublist] for item in sublist]
    return daily_discharges

## Длинные ряды в датафреймы

def to_dataframe(daily_discharges):
    DF_with_long_series = pd.DataFrame()
    DF_with_long_series['date'] = time_range_days
    DF_with_long_series['discharge'] = daily_discharges
    return DF_with_long_series

## Длинные ряды в месячные данные

def monthly_layers(finalized_discharge_data):
    monthly_discharges = [[] for _ in range(12)]
    for year in finalized_discharge_data:
        for i, month in enumerate(year):
            monthly_discharges[i].append(month)
    return monthly_discharges

## Группировка среднемесячных значений стока в датафреймы в лист по постам

def group_gauge_by_month(gauge_list):
    
    columns_of_data = gauges_by_month[0].columns[1:]
    monthly_dataframes = [[] for _ in range(len(columns_of_data))]
    for i, month in enumerate(columns_of_data):
        for gauge in gauge_list:
            monthly_dataframes[i].append(gauge[month])
                

    monthly_dataframes = [pd.DataFrame(gauge).T for gauge in monthly_dataframes]

    for gauge in monthly_dataframes:
        gauge.columns = WS_Names

    for gauge in monthly_dataframes:
        gauge.index = gauge_list[0].date
    return monthly_dataframes

## Кросс-корреляция между месяцами; график

def correlation_plot(data, plot_names, marker):
    """Input is the data frame"""

    plt.figure(figsize=(20, 10))

    from  matplotlib.colors import LinearSegmentedColormap
    cmap = LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N = 8)
    corr = data.corr()
    mask = np.triu(np.ones_like(corr, dtype=np.bool))
    ax = sns.heatmap(corr, annot=True, linewidths = 0.5, cmap = cmap, vmin = -1, vmax = 1, square = True, mask = mask)
    ax.set_title(plot_names)
    
    if marker == True:

        path_for_data = os.getcwd() + r'\PLOTS\''

        if not os.path.exists(path_for_data):
            os.makedirs(path_for_data)
        plt.savefig(path_for_data + '{}.png'.format(plot_names), bbox_inches = 'tight')
        return ax
    else:
        return ax

## Кросс-корреляция между месяцами - разница для разных периодов; график

def correlation_plot_difference(corr_1, corr_2, plot_names, marker):
    """Input is the correlation matrix"""
    from  matplotlib.colors import LinearSegmentedColormap
    plt.figure(figsize=(20, 10))
    cmap = LinearSegmentedColormap.from_list('rg',["g","y","r"], N = 8)
    mask = np.triu(np.ones_like(corr_2 - corr_1, dtype=np.bool))
    ax = sns.heatmap(abs(corr_2 - corr_1), annot=True, linewidths = 0.5, cmap = cmap, vmin = 0, vmax = 1, square = True, mask = mask)
    ax.set_title(plot_names)
    
    if marker == True:

        path_for_data = os.getcwd() + r'\PLOTS\''

        if not os.path.exists(path_for_data):
            os.makedirs(path_for_data)
        plt.savefig(path_for_data + '{}.png'.format(plot_names), bbox_inches = 'tight')
        return ax
    else:
        return ax

## Функция определения коэффициента детерминации

def rsquared(x, y):
    """ Return R^2 where x and y are array-like."""

    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

## Функция определения кривых спада

def decline_curves(gauge_array):
    """создание временного куска"""

    temp = gauge_array

    try_1 = [[temp.date.iloc[i], temp.discharge.iloc[i], True] if temp.discharge.iloc[i] < temp.discharge.iloc[i-1] else [temp.date.iloc[i], temp.discharge.iloc[i], False] for i in range(len(temp))]

    """если один элемент не удовлетворяют, это всё равно спад"""

    j = 0
    for i, check in enumerate(try_1):
        if try_1[i][2] == False:
            j += 1
            if j <= 1:
                try_1[i][2] = True
        else:
            j = 0

    """группировка по индексу True в условии спада"""

    from itertools import groupby
    import operator

    try_2 = [list(grp) for k, grp in groupby(try_1, operator.itemgetter(2))]

    """выделение только длинных ветвей спада"""

    try_3 = [i for i in try_2 if (len(i) >= 24) & (i[0][2] == True)]

    try_3_DF = [pd.DataFrame(i, columns = ['date', 'discharge', 'decline']) for i in try_3]

    winter_decline = [decline_line for decline_line in try_3_DF if (decline_line.date[0].month in [12, 1, 2])]
    
    spring_decline = [decline_line for decline_line in try_3_DF if (decline_line.date[0].month in [3, 4, 5])]
    
    summer_autumn_decline = [decline_line for decline_line in try_3_DF if (decline_line.date[0].month in [6, 7, 8, 9, 10, 11])]
    
    declines = [winter_decline, spring_decline, summer_autumn_decline]
    
    return declines

## Функция формирования списков кривых спада. Отрисовка графиков. Сезоны

def Plot_Decline_Lines(decline, j, i, ii, marker):

    def func_1(x, a, b, c):
        return -a * x ** b + c

    def func(x, a, b, c):
        return a * x ** b + c

    decline = decline.dropna()
    xdata = decline.index
    ydata = decline.discharge.values

    plot_names_for_season = [[season + WS_Names[i] for i in range(len(WS_Names))] for season in ['Зима ', 'Весна ', 'Лето-Осень ']]
    plot_name = 'Кривая #{} '.format(j+1) + plot_names_for_season[i][ii]
    xticks = list(pd.date_range(decline.date.min(), decline.date.max(), periods = 6))
    plt.figure(figsize=(20, 10))

    if np.abs(popt[1]) < 0.01:
        r2 = rsquared(func_1(ydata, popt[0], popt[1], popt[2]), ydata)
        plt.scatter(decline.date, ydata, color = 'b')
        plt.xticks(xticks, rotation=45)
        plt.plot(decline.date, func_1(xdata, *popt), 'r-',
                label='a=%5.3f, b=%5.3f, c=%5.3f, r2=%5.3f' % tuple(np.append(popt, r2)))
        plt.legend(loc=1, prop={'size': 24})
        plt.title(plot_name)
        if marker == True:

            path_for_data = os.getcwd() + r'/season_PLOTS/'

            if not os.path.exists(path_for_data):
                os.makedirs(path_for_data)
            plt.savefig(path_for_data + '{}.png'.format(plot_name), bbox_inches = 'tight')
        else:
            plt.show()
    else:
        r2 = rsquared(func(ydata, popt[0], popt[1], popt[2]), ydata)
        plt.scatter(decline.date, ydata, color = 'b')
        plt.xticks(xticks, rotation=45)
        plt.plot(decline.date, func(xdata, *popt), 'r-',
                label='a=%5.3f, b=%5.3f, c=%5.3f, r2=%5.3f' % tuple(np.append(popt, r2)))
        plt.legend(loc=1, prop={'size': 24})
        plt.title(plot_name)
        if marker == True:

            path_for_data = os.getcwd() + r'/season_PLOTS/'

            if not os.path.exists(path_for_data):
                os.makedirs(path_for_data)
            plt.savefig(path_for_data + '{}.png'.format(plot_name), bbox_inches = 'tight')
        else:
            plt.show()

## Функция формирования списков кривых спада. Отрисовка графиков. Подряд

def Plot_Decline_Lines_Long(decline, ii, marker):

    def func_1(x, a, b, c):
        return -a * x ** b + c

    def func(x, a, b, c):
        return a * x ** b + c

    decline = decline.dropna()
    xdata = decline.index
    ydata = decline.discharge.values

    plot_name = 'Кривая спада' + WS_Names[ii]
    xticks = list(pd.date_range(decline.date.min(), decline.date.max(), periods = 10))
    plt.figure(figsize=(20, 10))

    if np.abs(popt[1]) < 0.01:
        r2 = rsquared(func_1(ydata, popt[0], popt[1], popt[2]), ydata)
        plt.scatter(decline.date, ydata, color = 'b')
        plt.xticks(xticks, rotation=45)
        plt.plot(decline.date, func_1(xdata, *popt), 'r-',
                label='a=%5.3f, b=%5.3f, c=%5.3f, r2=%5.3f' % tuple(np.append(popt, r2)))
        plt.legend(loc=1, prop={'size': 24})
        plt.title(plot_name)
        if marker == True:

            path_for_data = os.getcwd() + r'/long_decline_PLOTS/'

            if not os.path.exists(path_for_data):
                os.makedirs(path_for_data)
            plt.savefig(path_for_data + '{}.png'.format(plot_name), bbox_inches = 'tight')
        else:
            pass
    else:
        r2 = rsquared(func(ydata, popt[0], popt[1], popt[2]), ydata)
        plt.scatter(decline.date, ydata, color = 'b')
        plt.xticks(xticks, rotation=45)
        plt.plot(decline.date, func(xdata, *popt), 'r-',
                label='a=%5.3f, b=%5.3f, c=%5.3f, r2=%5.3f' % tuple(np.append(popt, r2)))
        plt.legend(loc=1, prop={'size': 24})
        plt.title(plot_name)
        if marker == True:

            path_for_data = os.getcwd() + r'/long_decline_PLOTS/'

            if not os.path.exists(path_for_data):
                os.makedirs(path_for_data)
            plt.savefig(path_for_data + '{}.png'.format(plot_name), bbox_inches = 'tight')
        else:
            pass

## Функция формирования списков кривых спада. Подряд

def decline_curves_long(gauge_array):

    temp = gauge_array

    try_1 = [[temp.date.iloc[i], temp.discharge.iloc[i], True] if temp.discharge.iloc[i] < temp.discharge.iloc[i-1] else [temp.date.iloc[i], temp.discharge.iloc[i], False] for i in range(len(temp))]

    """если один элемент не удовлетворяют, это всё равно спад"""

    j = 0
    for i, check in enumerate(try_1):
        if try_1[i][2] == False:
            j += 1
            if j <= 1:
                try_1[i][2] = True
        else:
            j = 0

    """группировка по индексу True в условии спада"""

    from itertools import groupby
    import operator

    try_2 = [list(grp) for k, grp in groupby(try_1, operator.itemgetter(2))]

    """выделение только длинных ветвей спада"""

    try_3 = [i for i in try_2 if (len(i) >= 24) & (i[0][2] == True)]

    try_3_DF = [pd.DataFrame(i, columns = ['date', 'discharge', 'decline']) for i in try_3]

    return try_3_DF

## Функция определения статистик для рядов коэффициентов спада. Сезоны

def Stat_for_Decline(popt_power_list, column_names, i, j):

        popt_power_list = season      
        if len(popt_power_list) > 1:
            #mean
            MEAN_T = round(np.mean(popt_power_list), 2)

            import pymannkendall as mk # https://github.com/mmhs013/pyMannKendall

            #mk
            MK_T_005 = mk.original_test(popt_power_list, alpha=0.05)

            MK_T_01 = mk.original_test(popt_power_list, alpha=0.1)

            MK_T_trend_005 = MK_T_005[0]

            if MK_T_trend_005 == 'no trend':
                MK_T_trend_005 = '0'
            elif MK_T_trend_005 == 'increasing':
                MK_T_trend_005 = '1'
            else:
                MK_T_trend_005 = '-1'

            MK_T_trend_01 = MK_T_01[0]

            if MK_T_trend_01 == 'no trend':
                MK_T_trend_01 = '0'
            elif MK_T_trend_01 == 'increasing':
                MK_T_trend_01 = '1'
            else:
                MK_T_trend_01 = '-1'

            MK_T_p = MK_T_01[1]

            #spearman https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.spearmanr.html

            from scipy import stats

            SRC_T = stats.spearmanr(popt_power_list, range(len(popt_power_list)))

            SRC_T_rho_p = SRC_T[1]

            #THEIL-Sen https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.mstats.theilslopes.html

            THEIL_T = stats.theilslopes(popt_power_list)

            THEIL_T_S = round(THEIL_T[0], 3)

            #AR1
            def autocorr(x, t=1):
                return np.corrcoef(np.array([x[:-t], x[t:]]))[0][1]

            AR1_T = autocorr(popt_power_list)

            #TFPW

            TFPW_T_01 = np.array(mk.trend_free_pre_whitening_modification_test(popt_power_list, alpha=0.1))[[0, 2, 4, 7]]

            TFPW_T_S = float(TFPW_T_01[3])

            # Change in data

            if np.abs(AR1_T) < 0.2:
                CHANGE = round(THEIL_T_S * len(popt_power_list), 3)
                CHANGE_percent = round((THEIL_T_S * 100 * len(popt_power_list)) / abs(MEAN_T), 1)
            else:
                CHANGE = round(TFPW_T_S * len(popt_power_list), 3)
                CHANGE_percent = round((TFPW_T_S * 100 * len(popt_power_list)) / abs(MEAN_T), 1)

            index_names = ['Всего кривых', 'Среднее', 'Манн-Кендалл(p<0.05)',
                        'Манн-Кендалл(p<0.1)', 'Тэйл-Сен', 'Изменение', 'Изменение, %']
            power_index_DF = pd.DataFrame([len(popt_power_list), MEAN_T,
                                        MK_T_trend_005, MK_T_trend_01, THEIL_T_S,
                                        CHANGE, CHANGE_percent], index = index_names, columns = [column_names[i][j]])
            return power_index_DF
        else:
            index_names = ['Всего кривых', 'Среднее', 'Манн-Кендалл(p<0.05)',
                        'Манн-Кендалл(p<0.1)', 'Тэйл-Сен', 'Изменение', 'Изменение, %']
            power_index_DF = pd.DataFrame([len(popt_power_list), np.NaN,
                                        np.NaN, np.NaN, np.NaN,
                                        np.NaN, np.NaN], index = index_names, columns = [column_names[i][j]])
            return power_index_DF

## Функция определения статистик для рядов коэффициентов спада. Подряд

def Stat_for_Decline_long(popt_power_list, column_names, i):

        if len(popt_power_list) > 1:
            #mean
            MEAN_T = round(np.mean(popt_power_list), 2)

            import pymannkendall as mk # https://github.com/mmhs013/pyMannKendall

            #mk
            MK_T_005 = mk.original_test(popt_power_list, alpha=0.05)

            MK_T_01 = mk.original_test(popt_power_list, alpha=0.1)

            MK_T_trend_005 = MK_T_005[0]

            if MK_T_trend_005 == 'no trend':
                MK_T_trend_005 = '0'
            elif MK_T_trend_005 == 'increasing':
                MK_T_trend_005 = '1'
            else:
                MK_T_trend_005 = '-1'

            MK_T_trend_01 = MK_T_01[0]

            if MK_T_trend_01 == 'no trend':
                MK_T_trend_01 = '0'
            elif MK_T_trend_01 == 'increasing':
                MK_T_trend_01 = '1'
            else:
                MK_T_trend_01 = '-1'

            MK_T_p = MK_T_01[1]

            #spearman https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.spearmanr.html

            from scipy import stats

            SRC_T = stats.spearmanr(popt_power_list, range(len(popt_power_list)))

            SRC_T_rho_p = SRC_T[1]

            #THEIL-Sen https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.mstats.theilslopes.html

            THEIL_T = stats.theilslopes(popt_power_list)

            THEIL_T_S = round(THEIL_T[0], 3)

            #AR1
            def autocorr(x, t=1):
                return np.corrcoef(np.array([x[:-t], x[t:]]))[0][1]

            AR1_T = autocorr(popt_power_list)

            #TFPW

            TFPW_T_01 = np.array(mk.trend_free_pre_whitening_modification_test(popt_power_list, alpha=0.1))[[0, 2, 4, 7]]

            TFPW_T_S = float(TFPW_T_01[3])

            # Change in data

            if np.abs(AR1_T) < 0.2:
                CHANGE = round(THEIL_T_S * len(popt_power_list), 3)
                CHANGE_percent = round((THEIL_T_S * 100 * len(popt_power_list)) / abs(MEAN_T), 1)
            else:
                CHANGE = round(TFPW_T_S * len(popt_power_list), 3)
                CHANGE_percent = round((TFPW_T_S * 100 * len(popt_power_list)) / abs(MEAN_T), 1)

            index_names = ['Всего кривых', 'Среднее', 'Манн-Кендалл(p<0.05)',
                        'Манн-Кендалл(p<0.1)', 'Тэйл-Сен', 'Изменение', 'Изменение, %']
            power_index_DF = pd.DataFrame([len(popt_power_list), MEAN_T,
                                        MK_T_trend_005, MK_T_trend_01, THEIL_T_S,
                                        CHANGE, CHANGE_percent], index = index_names, columns = [column_names[i]])
            return power_index_DF
        else:
            index_names = ['Всего кривых', 'Среднее', 'Манн-Кендалл(p<0.05)',
                        'Манн-Кендалл(p<0.1)', 'Тэйл-Сен', 'Изменение', 'Изменение, %']
            power_index_DF = pd.DataFrame([len(popt_power_list), np.NaN,
                                        np.NaN, np.NaN, np.NaN,
                                        np.NaN, np.NaN], index = index_names, columns = [column_names[i]])
            return power_index_DF



### Чтение базы в заказанном формате

In [ ]:
cols = pd.read_excel(my_databse, skiprows=2, 
                     usecols=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], skipfooter=999)

gauge_list = [pd.read_excel(my_databse, sheet_name = i, skipfooter=0, skip_blank_lines=False) for i in WS_Names]

gauge_list = [from_df_to_discharge_chaos(i) for i in gauge_list]

time_range_days = pd.date_range(start='1/1/1970', end='31/12/2016')
time_range_monthes = pd.date_range(start='1/1/1970', end='31/12/2016',  freq='MS')
time_range_years = pd.date_range(start='1/1/1970', end='31/12/2016',  freq='Y')
monthes_for_years = [[] for _ in time_range_years]

## Разбиение списка месяцев, на список списков месяцев по годам

for i in enumerate(time_range_years):
    for j in enumerate(time_range_monthes):
        fnd = True
        while fnd:
            fnd = False
            if i[1].year == j[1].year:
                monthes_for_years[i[0]].append(j[1])
                fnd = True
                break

gauge_list = [make_monthes_cute(i) for i in gauge_list]

## В слои стока
gauge_list = [[[[(day * 10**3 / WS_area_LIST[i]) for day in month] for month in year] for year in gauge] for i, gauge in enumerate(gauge_list)]

## Длинные ряды
gauge_list_long_series = [to_long_series(i) for i in gauge_list]

## Длинные ряды в датафреймы
gauge_list_long_series = [to_dataframe(i) for i in gauge_list_long_series]

## В список датафреймов постов со среднемесячными слоями

monthes = ['Январь','Февраль','Март','Апрель','Май','Июнь','Июль','Август','Сентябрь','Октябрь','Ноябрь', 'Декабрь']
monthes_series = [monthly_layers(i) for i in gauge_list]

layer_series = [[[np.mean(month) for month in year] for year in gauge] for i, gauge in enumerate(monthes_series)]

gauges_by_month = list()
for gauge in layer_series:
    gauges_by_month.append(pd.DataFrame(gauge, index = monthes).T)

for i, gauge in enumerate(gauges_by_month):
    gauge.insert(loc = 0, column = 'date', value = time_range_years.year)
    gauge.set_index(['date'], inplace=True)
    gauge['Среднегодовое'] = gauge[monthes].mean(axis = 1, skipna = False)
    gauge = gauge.reset_index(drop = True)

for gauge in gauges_by_month:
    for columns in gauge.columns:
        gauge[columns] = gauge[columns].astype('float64').apply('{:.2f}'.format)
        gauge[columns] = gauge[columns].astype('float64')



### Формат для анализа

In [ ]:
for i, gauge in enumerate(gauges_by_month):
    gauge.insert(loc = 0, column = 'date', value = time_range_years.year)
    gauges_by_month[i] = gauges_by_month[i].reset_index(drop = True)

### Разбиение на периоды

In [ ]:
gauges_by_month_1986 = [gauge[gauge.date <= 1986] for gauge in gauges_by_month]
gauges_by_month_2000 = [gauge[(gauge.date > 1986) & (gauge.date < 2000)] for gauge in gauges_by_month]
gauges_by_month_2016 = [gauge[gauge.date >= 2000] for gauge in gauges_by_month]

gauges_by_month_1986 = group_gauge_by_month(gauges_by_month_1986)
gauges_by_month_2000 = group_gauge_by_month(gauges_by_month_2000)
gauges_by_month_2016 = group_gauge_by_month(gauges_by_month_2016)
gauge_periods = [gauges_by_month_1986, gauges_by_month_2000, gauges_by_month_2016]

## Подписи к графикам 

columns_of_data = gauges_by_month[0].columns[1:]
labels_for_1986 = list()
for columns in columns_of_data:
    labels_for_1986.append(columns + ' за 1970-1986')

labels_for_2000 = list()
for columns in columns_of_data:
    labels_for_2000.append(columns + ' за 1987-2001')

labels_for_2016 = list()
for columns in columns_of_data:
    labels_for_2016.append(columns + ' за 2002-2016')

labels_for_periods = [labels_for_1986, labels_for_2000, labels_for_2016]

### График корреляционной функции. Помесячная

In [ ]:
for j, period in enumerate(gauge_periods):
    for i, gauge in enumerate(period):
        correlation_plot(gauge, labels_for_periods[j][i], False)

### Дендрограмма для классификации постов

In [ ]:
plt.figure(figsize=(15, 15))
dend = shc.dendrogram(shc.linkage(gauges_by_month_1986[12].dropna().T,
                                  method='ward'), labels = WS_Names)
plt.xticks(rotation=90)
title = 'Дендрограмма распределения постов'
plt.title(title)

path_for_data = os.getcwd() + r'\PLOTS\''
if not os.path.exists(path_for_data):
    os.makedirs(path_for_data)

plt.savefig(path_for_data + '{}.png'.format(title), bbox_inches = 'tight')

### Корреляционная разность постов за разные периоды

In [ ]:
correlation_plot_difference(gauges_by_month_2016[12].corr(), gauges_by_month_1986[12].corr(), 'Абсолютная разница корреляции за последний и первый периоды', True)

### Разбиение на длинные периоды модулей подряд

In [ ]:
gauges_long_1986 = [gauge[gauge.date <= '1986'] for gauge in gauge_list_long_series]
temp = [gauge.discharge.values for gauge in gauges_long_1986]
gauges_long_1986 = pd.DataFrame(temp, columns = gauges_long_1986[0].date, index = WS_Names)
gauges_long_1986 = gauges_long_1986.T

gauges_long_2000 = [gauge[(gauge.date > '1986') & (gauge.date < '2002')] for gauge in gauge_list_long_series]
temp = [gauge.discharge.values for gauge in gauges_long_2000]
gauges_long_2000 = pd.DataFrame(temp, columns = gauges_long_2000[0].date, index = WS_Names)
gauges_long_2000 = gauges_long_2000.T

gauges_long_2016 = [gauge[gauge.date >= '2002'] for gauge in gauge_list_long_series]
temp = [gauge.discharge.values for gauge in gauges_long_2016]
gauges_long_2016 = pd.DataFrame(temp, columns = gauges_long_2016[0].date, index = WS_Names)
gauges_long_2016 = gauges_long_2016.T

long_periods = [gauges_long_1986, gauges_long_2000, gauges_long_2016]
long_periods_labels = ['1970-1986', '1987-2001', '2002-2016']

### График корреляционной функции. Подряд

In [ ]:
for i, period in enumerate(long_periods):
    correlation_plot(period, long_periods_labels[i], False)

### Статистическая обработка данных

In [ ]:
from izhora_class import izhora_class

In [ ]:
izhora_class(gauges_by_month, WS_Names, 'Izhora_Flow')

### Формирование рядов коэффициентов спада. Сезоны

In [ ]:
plot_names_for_season = [[season + WS_Names[i] for i in range(len(WS_Names))] for season in ['Зима ', 'Весна ', 'Лето-Осень ']]

winter_coefficients = list()
spring_coefficients = list()
summer_autumn_coefficients = list()
power_coefficients = [[winter_coefficients, spring_coefficients, summer_autumn_coefficients] for _ in range(len(gauge_list_long_series))]

def func_1(x, a, b, c):
    return -a * x ** b + c

def func(x, a, b, c):
    return a * x ** b + c

for ii in trange(len(gauge_list_long_series)):

    declines = decline_curves(gauge_list_long_series[ii])

    """создание листа коэффициентов спада. Графики"""

    for i, season in enumerate(declines):
        
        power_coefficients[ii][i] = list()

        for j, decline in enumerate(season):
            
            decline = decline.dropna()
            xdata = decline.index
            ydata = decline.discharge.values

            popt, pcov = curve_fit(func, xdata, ydata, maxfev = 2000)

            if np.abs(popt[1]) < 0.01:

                Plot_Decline_Lines(decline, j, i, ii, True)
                popt, pcov = curve_fit(func_1, xdata, ydata, maxfev = 2000)
                power_coefficients[ii][i].append(popt[1])

            else:

                Plot_Decline_Lines(decline, j, i, ii, True)
                popt, pcov = curve_fit(func, xdata, ydata, maxfev = 2000)
                power_coefficients[ii][i].append(popt[1])

### Формирование рядов коэффициентов спада. Подряд

In [ ]:
plot_names_for_season = ['Длинный спад' + WS_Names[i] for i in range(len(WS_Names))]

power_coefficients_long = [[] for _ in range(len(gauge_list_long_series))]

for ii in trange(len(gauge_list_long_series)):

    declines = decline_curves_long(gauge_list_long_series[ii])

    """создание листа коэффициентов спада Длинный ряд. Графики"""
        
    power_coefficients_long[ii] = list()

    for decline in declines:
            
        decline = decline.dropna()
        xdata = decline.index
        ydata = decline.discharge.values

        popt, pcov = curve_fit(func, xdata, ydata, maxfev = 2000)

        if np.abs(popt[1]) < 0.01:
            popt, pcov = curve_fit(func_1, xdata, ydata, maxfev = 2000)
            power_coefficients_long[ii].append(popt[1])

        else:
            popt, pcov = curve_fit(func, xdata, ydata, maxfev = 2000)
            power_coefficients_long[ii].append(popt[1])

### Сохранение данных

In [ ]:
path_for_data = os.getcwd() + r'/long_decline_STAT/'

column_names = ['Коэффициенты спада {}'.format(WS_Names[i]) for i in range(len(WS_Names))]
season_stack_long = list()

for i, gauge in enumerate(power_coefficients_long):

    season_stack_long.append(Stat_for_Decline_long(gauge, column_names, i))

if not os.path.exists(path_for_data):
    os.makedirs(path_for_data)

for i,j in enumerate(season_stack_long):
    j.to_csv(path_for_data + '\{}.txt'.format(WS_Names[i] + ' спад'), sep="\t", index = True)